In [58]:
import os
from transformers import BertConfig, BertTokenizer, BertModel

class BertEmbeddingsDK(object):
    r"""Class for creating Danish BERT word and sentence on the fly."""
    
    def __init__(self, bert_path=None):
        
        
        self.url = 'https://www.dropbox.com/s/19cjaoqvv2jicq9/danish_bert_uncased_v2.zip?dl=1'
        if bert_path == None: 
            self.bert_path = './bert-base-danish/'
        else: 
            self.bert_path = bert_path        
            
        # Handle model
        if os.path.isfile(self.bert_path + 'pytorch_model.bin'):
            _load_model()
        elif # ALL MODEL FILES IN TF:
            # convert model to pytorch
        else:
            # download
            # osv.
            
            
        
        
            
    def _load_model(self):
        self.config = BertConfig.from_json_file(self.bert_path+'config.json')
        self.tokenizer = BertTokenizer(vocab_file=self.bert_path+'vocab.txt')
        self.model = BertModel(config=self.config).from_pretrained(self.bert_path)
        
    
    def create_embedding(self, text, print_num_tokens=False):
        
        tokenized = self.tokenizer(text, return_tensors='pt')
        if print_num_tokens: print(tokenized['input_ids'].shape)
        embedding = self.model(**tokenized, output_hidden_states=True)
        
        return embedding
        
        
        
embedder = BertEmbeddingsDK()#bert_path='/home/juunge/models/bert-base-danish/')

In [59]:
text = 'Igen en sober og saglig dialog. I lytter til gæsten og I underspiller ikke det komplekse i, at deltagelse i en bæredygtig rejse kun kræver omtanke og et valg.... helt nede i de små detaljer. Tøv ikke - Bare få begyndt! Og så er der udfordringerne ... tankevækkende at lytte med på'
embedding = embedder.create_embedding(text, print_num_tokens=True)

torch.Size([1, 69])


In [48]:
print('Output lenght:', len(embedding))

print('\nDim1 shape:', embedding[0].shape)

print('\nDim2 shape:', embedding[1].shape)

print('\n\nDim3 length:',len(embedding[2]))
print('Dim3, Dimn shape:', embedding[2][12].shape)


# Hidden layers out[2][0-12]

Output lenght: 3

Dim1 shape: torch.Size([1, 69, 768])

Dim2 shape: torch.Size([1, 768])


Dim3 length: 13
Dim3, Dimn shape: torch.Size([1, 69, 768])


## Download and convert Danish BERT

In [49]:
import requests
import os

# Download Danish Bert
url = 'https://www.dropbox.com/s/19cjaoqvv2jicq9/danish_bert_uncased_v2.zip?dl=1'
r = requests.get(url, allow_redirects=True)
open('bert-base-danish.zip', 'wb').write(r.content)


os.system('unzip bert-base-danish.zip')
os.system('mv danish_bert_uncased_v2 bert-base-danish')
os.system('rm bert-base-danish.zip')
os.system('rm -rf __MACOSX')
os.system('transformers-cli convert --model_type bert --tf_checkpoint ./bert-base-danish/bert_model.ckpt --config ./bert-base-danish/bert_config.json --pytorch_dump_output ./bert-base-danish/pytorch_model.bin')
os.system('mv bert-base-danish/bert_config.json bert-base-danish/config.json')


## Get danish BERT on linux:
    # 1. download           --> wget https://www.dropbox.com/s/19cjaoqvv2jicq9/danish_bert_uncased_v2.zip?dl=1
    # 2. unzip              --> unzip danish_bert_uncased_v2.zip?dl=1
    # 3. rm zipped          --> rm danish_bert_uncased_v2.zip\?dl\=1
    # 4. rename             --> mv danish_bert_uncased_v2 bert-base-danish
    # 5. convert to pytorch --> transformers-cli convert --model_type bert --tf_checkpoint ./bert-base-danish/bert_model.ckpt --config ./bert-base-danish/bert_config.json --pytorch_dump_output ./bert-base-danish/pytorch_model.bin
    # 6. rename config      --> mv bert-base-danish/bert_config.json bert-base-danish/config.json    
    

1222631135